In [21]:
import os
from pathlib import Path
import xml.etree.ElementTree as ET

from tqdm import tqdm
import numpy as np
import math
import random

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt

import cv2

from models_class import BIG_BED, OPTIM_BED
from utils import (
    yolo2pixel,
    nms_yv1,
    get_bboxes,
    mAP,
    plot_preds)

# Folders

In [22]:
ds_dir = '../../ds2fire/'

### DFire

In [23]:
dfire_dir = ds_dir + 'dfire_yolo/'
dfire_val_dir = dfire_dir + 'test/'
dfire_val_imgs = dfire_val_dir + 'images/'
dfire_val_labels = dfire_val_dir + 'labels/'

print(f'DFire val dir: {os.listdir(dfire_val_dir)}')

DFire val dir: ['.ipynb_checkpoints', 'labels', 'images']


### DFS

In [24]:
dfs_dir = ds_dir + 'dfs_xml/'
dfs_val_imgs = dfs_dir + 'images/'
dfs_val_labels = dfs_dir + 'labels/'

print(f'DFS val dir: {os.listdir(dfs_dir)}')

DFS val dir: ['labels', 'images']


### DFS Files: seed to replicate training split files

In [25]:
# Get all txt file paths in path_annot and sort them
dfs_xml_files = sorted(
    [
        os.path.join(dfs_val_labels, file_name)
        for file_name in os.listdir(dfs_val_labels)
        if file_name.endswith(".xml")
    ]
)

dfs_len = len(dfs_xml_files)
dfs_train_elements = int(dfs_len*0.8)
dfs_test_elements = dfs_len - dfs_train_elements

random.seed(123)
random.shuffle(dfs_xml_files)
dfs_test_list = dfs_xml_files[dfs_train_elements:]

print(f'DFS number of samples: {dfs_len}')
print(f'DFS test samples: {len(dfs_test_list)}')

DFS number of samples: 9462
DFS test samples: 1893


# Config

In [26]:
CLASSES = ["smoke", "fire"]

IMG_DIM = {'W':224, 'H':224} # (W, H)
IMG_W = IMG_DIM['W']
IMG_H = IMG_DIM['H']

SX = 7
SY = 7
B = 2 # Number of bounding boxes to predict.
C = len(CLASSES) # Number of classes in the dataset.

BATCH_SIZE = 64
WORKERS = 2

# DFire Dataset

In [27]:
class DFireDataset(Dataset):
    '''
    Creates a Pytorch Dataset to train the Yolov1 Network.
    Encodes labels to match the format [xcell, ycell, w, h, confidence, class_0 (smoke), class_1 (fire)]
        - Final encoding format is: [xcell, ycell, w, h, conf=1, smoke?, fire?]

    Discard images when there are more than 1 object in the same cell
    
    Arguments:
        - img_h:            image height
        - img_w:            image width
        - img_dir:          path to images folder
        - label_dir:        path to labels folder
        - SX:               number of cells in X axis (horizontal -> width)
        - SY:               number of cells in Y axis (vertical -> height)
        - C:                number of classes, 2 in this case
        - transform:        transformation applied to input images -> Albumentations
        - target_transform: transformation applied to labels -> nothing by default

    Return:
        - img:              1 image of the dataset
        - target:           corresponding label encoded
    '''

    def __init__(self, img_h, img_w, img_dir, label_dir, 
                 SX, SY, C, 
                 transform=None, target_transform=None):
        self.img_h = img_h
        self.img_w = img_w
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.SX = SX
        self.SY = SY
        self.C = C
        self.transform = transform
        self.target_transform = target_transform

        self.labels_list = sorted(
            [
                os.path.join(self.label_dir, file_name)
                for file_name in os.listdir(self.label_dir)
                if file_name.endswith(".txt")
            ]
        )
        
        self.images, self.bboxes, self.labels = self.__build_ds__(self.labels_list)
        
        self.num_samples = self.images.shape[0]

    def __len__(self):
        return self.num_samples
    
    def __bbox_check__(self, bbox):
        eps = 1e-6
        
        xc, yc, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
        xmin = xc - w/2
        ymin = yc - h/2
        xmax = xc + w/2
        ymax = yc + h/2
        
        xmin = max(xmin, 0 + eps)
        ymin = max(ymin, 0 + eps)
        xmax = min(xmax, 1)
        ymax = min(ymax, 1)
        
        bbox = np.array([ 
                (xmin+xmax)/2,
                (ymin+ymax)/2,
                xmax-xmin,
                ymax-ymin
                 ]).astype(np.float32)
        
        return bbox        


    def __build_ds__(self, labels_list):
        bboxes = []
        labels = []
        images = []
        wrong_imgs = 0
        overlapping_rem = 0
        more_than_3 = 0
                
        for label in labels_list:
            fname = Path(label).stem
            image_path = self.img_dir + fname + '.jpg'   
            #print(fname, image_path)
                                   
            if cv2.imread(image_path) is None:
                print(f'{image_path} cannot be read by cv2 -> removed')
                wrong_imgs += 1
            
            else:
                
                label_mtx = np.zeros((self.SY, self.SX))
                overlapping_object = 0

                one_bboxes = []
                one_labels = []
            
                with open(label) as f:
                    lines = f.readlines()

                    # Restrict to 3 boxes per sample
                    if len(lines) > 3:
                        more_than_3 += 1
                        continue
                        
                    for line in lines:
                        class_id, x, y, w, h = line.strip().split()
                        class_id = int(class_id)
                        box = np.array([x, y, w, h]).astype(np.float32)
                        x, y, w, h = box[0], box[1], box[2], box[3]
                        box_ok = self.__bbox_check__([x, y, w, h])
                        x, y, w, h = box_ok[0], box_ok[1], box_ok[2], box_ok[3]
                        i, j = math.floor(y * self.SY), math.floor(x * self.SX)
                        if label_mtx[i, j] == 1:
                            overlapping_object = 1
                            overlapping_rem += 1
                            #print(f'Removed {label} due to overlapping object in cell {i, j}')
                            break
                        else:
                            label_mtx[i, j] = 1
                            one_bboxes.append([x, y, w, h])
                            # smoke
                            if class_id == 0:
                                one_labels.append(0)
                            # fire
                            elif class_id == 1:
                                one_labels.append(1)
                            else:
                                print(f'File {label} errored in cell {i, j}')

                    if overlapping_object == 0:
                        # Padding to SX*SY labels and bounding boxes, so you can store tensors
                        # Label -1 indicates no box
                        for idx in range(self.SX*self.SY - len(one_labels)):
                            one_bboxes.append([0, 0, 0, 0])
                            one_labels.append(-1)
                        # print(f'\nBboxes and Labels of image {image_path}')
                        # print("Bboxes")
                        # for box in one_bboxes:
                        #     print(box)
                        # print("Labels")
                        # for label in one_labels:
                        #     print(label)
                        bboxes.append(one_bboxes)
                        labels.append(one_labels)
                        images.append(image_path)
        
        print(f'Removed wrong images: {wrong_imgs}')
        print(f'Removed due to overlapping: {overlapping_rem}')
        print(f'Removed due to more than 3: {more_than_3}')

        labels_np = np.array(labels)
        labels_tensor = torch.tensor(labels_np, dtype=torch.float32)
        bboxes_np = np.array(bboxes)
        bboxes_tensor = torch.tensor(bboxes_np, dtype=torch.float32)
        images_array = np.array(images)
        # print(f'Images array {images_array}')
        # print(f'Bboxes tensor {bboxes_tensor}')
        # print(f'Labels tensor {labels_tensor}')
        
        return images_array, bboxes_tensor, labels_tensor
        #return images, bboxes, labels

    def __getitem__(self, index):

        # Image processing
        img_file = self.images[index]
        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        #img = cv2.resize(img, (self.img_w, self.img_h), interpolation = cv2.INTER_NEAREST)

        # Labels processing
        bboxes = self.bboxes[index]
        bboxes = bboxes[~torch.all(bboxes == torch.tensor([0,0,0,0]), dim=1)]
        bboxes = bboxes.numpy().tolist()
        #print(bboxes)
        labels = self.labels[index]
        labels = labels[labels != -1.]
        labels = labels.numpy().tolist()
        #print(f'Labels inside dataset {labels}')
        
        # Data Augmentation
        if self.transform is not None:
            try:
                aug = self.transform(image=img, bboxes=bboxes, class_labels=labels)
                img = aug['image']
                bboxes = aug['bboxes']
                labels = aug['class_labels']
            except:
                #print(f'Error trying to augment image {img_file}')
                img = cv2.resize(img, (self.img_w, self.img_h), interpolation = cv2.INTER_NEAREST)
                img = (img / 255.) - 0.5 # Ocurrencia de flipao
                img = torch.tensor(img, dtype=torch.float32)
                img = img.permute(2, 0, 1)
        
        label_mtx = np.zeros((self.SY, self.SX, 5+self.C))
        
        for box, label in zip(bboxes, labels):
            class_id = int(label)
            i, j = int(box[1]*self.SY), int(box[0]*self.SX)
            xcell, ycell = box[0]*self.SX - j, box[1]*self.SY - i
            label_mtx[i, j, :5] = [xcell, ycell, box[2], box[3], 1]
            label_mtx[i, j, 5+class_id] = 1

        label_mtx = torch.tensor(label_mtx, dtype=torch.float32)
        
        #return img, label_mtx, img_file
        return img, label_mtx

# DFS Dataset

In [28]:
class DFSDataset(Dataset):
    '''
    Creates a Pytorch Dataset to train the Yolov1 Network.
    Encodes labels to match the format [xcell, ycell, w, h, confidence, class_0 (smoke), class_1 (fire)]
        - Final encoding format is: [xcell, ycell, w, h, conf=1, smoke?, fire?]

    Discard images when there are more than 1 object in the same cell
    
    Arguments:
        - img_h:            image height
        - img_w:            image width
        - img_dir:          path to images folder
        - label_dir:        path to labels folder
        - SX:               number of cells in X axis (horizontal -> width)
        - SY:               number of cells in Y axis (vertical -> height)
        - C:                number of classes, 2 in this case
        - transform:        transformation applied to input images -> Albumentations
        - target_transform: transformation applied to labels -> nothing by default

    Return:
        - img:              1 image of the dataset
        - target:           corresponding label encoded
    '''

    def __init__(self, img_h, img_w, img_dir, labels_list, 
                 SX, SY, C, 
                 transform=None, target_transform=None):
        self.img_h = img_h
        self.img_w = img_w
        self.img_dir = img_dir
        self.labels_list = labels_list
        self.SX = SX
        self.SY = SY
        self.C = C
        self.transform = transform
        self.target_transform = target_transform
        
        self.images, self.bboxes, self.labels = self.__build_ds__(self.labels_list)
        
        self.num_samples = self.images.shape[0]

    def __len__(self):
        return self.num_samples
    
    def __bbox_check__(self, bbox):
        eps = 1e-6
        
        xmin, ymin, xmax, ymax = bbox[0], bbox[1], bbox[2], bbox[3]
        
        xmin = max(xmin, 0 + eps)
        ymin = max(ymin, 0 + eps)
        xmax = min(xmax, 1)
        ymax = min(ymax, 1)
        
        bbox = np.array([ 
                (xmin+xmax)/2,
                (ymin+ymax)/2,
                xmax-xmin,
                ymax-ymin
                 ]).astype(np.float32)
        
        return bbox        


    def __build_ds__(self, labels_list):
        bboxes = []
        labels = []
        images = []
        wrong_imgs = 0
        overlapping_rem = 0
        more_than_3 = 0
        
        
        for xml_file in labels_list:
#             fname = Path(xml_file).stem
#             image_path = self.img_dir + fname + '.jpg'   
            #print(fname, image_path)
            
            tree = ET.parse(xml_file)
            root = tree.getroot()

            image_name = root.find("filename").text
            image_path = os.path.join(self.img_dir, image_name)
            #print(image_name, image_path)
            
            if cv2.imread(image_path) is None:
                print(f'{image_path} cannot be read by cv2 -> removed')
                wrong_imgs += 1
            
            else:
                overlapping_object = 0
                more_than_3_in = 0

                label_mtx = np.zeros((self.SY, self.SX))

                size = root.find("size")
                img_w = float(size.find("width").text)
                img_h = float(size.find("height").text)

                one_bboxes = []
                one_labels = []

                for obj in root.iter("object"):
                    class_name = obj.find("name").text
                    if class_name == 'smoke':
                        class_id = 0
                        more_than_3_in += 1
                    elif class_name == 'fire':
                        class_id = 1
                        more_than_3_in += 1
                    else:
                        continue 
                    bbox = obj.find("bndbox")
                    xmin = float(bbox.find("xmin").text)/img_w
                    ymin = float(bbox.find("ymin").text)/img_h
                    xmax = float(bbox.find("xmax").text)/img_w
                    ymax = float(bbox.find("ymax").text)/img_h

                    box = self.__bbox_check__([xmin, ymin, xmax, ymax])
                    #print(f'Class: {class_name} - Class_id: {class_id}. Coords: x={x}, y={y}, w={w}, h={h}')

                    x, y, w, h = box[0], box[1], box[2], box[3]
                    i, j = math.floor(y * self.SY), math.floor(x * self.SX)
                    if label_mtx[i, j] == 1:
                        overlapping_object = 1
                        overlapping_rem += 1
                        break
                    else:
                        label_mtx[i, j] = 1
                        one_bboxes.append([x, y, w, h])
                        # smoke
                        if class_id == 0:
                            one_labels.append(0)
                        # fire
                        elif class_id == 1:
                            one_labels.append(1)
                        else:
                            print(f'File {label} errored in cell {i, j}') 
                
                if more_than_3_in > 3:
                    more_than_3 += 1
                    continue
                
                if (overlapping_object == 0) and (more_than_3_in < 4):
                    # Padding to SX*SY labels and bounding boxes, so you can store tensors
                    # Label -1 indicates no box
                    for idx in range(self.SX*self.SY - len(one_labels)):
                        one_bboxes.append([0, 0, 0, 0])
                        one_labels.append(-1)
                    # print(f'\nBboxes and Labels of image {image_path}')
                    # print("Bboxes")
                    # for box in one_bboxes:
                    #     print(box)
                    # print("Labels")
                    # for label in one_labels:
                    #     print(label)
                    bboxes.append(one_bboxes)
                    labels.append(one_labels)
                    images.append(image_path)

        print(f'Removed wrong images: {wrong_imgs}')
        print(f'Removed due to overlapping: {overlapping_rem}')
        print(f'Removed due to more than 3: {more_than_3}')

        labels_np = np.array(labels)
        labels_tensor = torch.tensor(labels_np, dtype=torch.float32)
        bboxes_np = np.array(bboxes)
        bboxes_tensor = torch.tensor(bboxes_np, dtype=torch.float32)
        images_array = np.array(images)
        # print(f'Images array {images_array}')
        # print(f'Bboxes tensor {bboxes_tensor}')
        # print(f'Labels tensor {labels_tensor}')
        
        return images_array, bboxes_tensor, labels_tensor
        #return images, bboxes, labels


 
    def __getitem__(self, index):

        # Image processing
        img_file = self.images[index]
        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)   
        #img = cv2.resize(img, (self.img_w, self.img_h), interpolation = cv2.INTER_NEAREST)

        # Labels processing
        bboxes = self.bboxes[index]
        bboxes = bboxes[~torch.all(bboxes == torch.tensor([0,0,0,0]), dim=1)]
        bboxes = bboxes.numpy().tolist()
        #print(bboxes)
        labels = self.labels[index]
        labels = labels[labels != -1.]
        labels = labels.numpy().tolist()
        #print(f'Labels inside dataset {labels}')
        
        # Data Augmentation
        if self.transform is not None:
            try:
                aug = self.transform(image=img, bboxes=bboxes, class_labels=labels)
                img = aug['image']
                bboxes = aug['bboxes']
                labels = aug['class_labels']
            except:
                #print(f'Error trying to augment image {img_file}')
                img = cv2.resize(img, (self.img_w, self.img_h), interpolation = cv2.INTER_NEAREST)
                img = (img / 255.) - 0.5 # Ocurrencia de flipao
                img = torch.tensor(img, dtype=torch.float32)
                img = img.permute(2, 0, 1)
        
        label_mtx = np.zeros((self.SY, self.SX, 5+self.C))
        
        for box, label in zip(bboxes, labels):
            class_id = int(label)
            i, j = int(box[1]*self.SY), int(box[0]*self.SX)
            xcell, ycell = box[0]*self.SX - j, box[1]*self.SY - i
            label_mtx[i, j, :5] = [xcell, ycell, box[2], box[3], 1]
            label_mtx[i, j, 5+class_id] = 1

        label_mtx = torch.tensor(label_mtx, dtype=torch.float32)
        
        #return img, label_mtx, img_file
        return img, label_mtx

# Preprocess, Datasets and DataLoaders

In [29]:
val_transform = A.Compose([
    A.Resize(IMG_H, IMG_W, p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), p=1),
    ToTensorV2(p=1),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

dfire_val_dataset = DFireDataset(img_h = IMG_H,
                                 img_w = IMG_W,
                                 img_dir = dfire_val_imgs,
                                 label_dir = dfire_val_labels,
                                 SX = SX,
                                 SY = SY,
                                 C = C,
                                 transform=val_transform)

dfs_val_dataset = DFSDataset(img_h = IMG_H,
                             img_w = IMG_W,
                             img_dir = dfs_val_imgs,
                             labels_list = dfs_test_list,
                             SX = SX,
                             SY = SY,
                             C = C,
                             transform=val_transform)

full_test_ds = torch.utils.data.ConcatDataset((dfire_val_dataset, dfs_val_dataset))
print(f'Test dataset len: {len(full_test_ds)}')

val_loader = DataLoader(dataset=full_test_ds,
                        batch_size=BATCH_SIZE,
                        num_workers=WORKERS,
                        pin_memory=True,
                        shuffle=False,
                        drop_last=True)

Removed wrong images: 0
Removed due to overlapping: 118
Removed due to more than 3: 445
Removed wrong images: 0
Removed due to overlapping: 58
Removed due to more than 3: 277
Test dataset len: 5330


# Model

In [30]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


In [31]:
model = OPTIM_BED(num_classes=C, S=SX, B=B, in_channels=3).to(DEVICE)
model_path = '../../bed_pytorch_results/optim/'
model_file = model_path + 'bed_best_2BB_mAP.pt'

checkpoint = torch.load(model_file, map_location=torch.device(DEVICE))
model.load_state_dict(checkpoint['model_state_dict'])

model.eval() 

OPTIM_BED(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(24, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (9): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (10): ReLU()
    (11): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (12): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    (13): ReLU()
    (14): Conv2d(32, 32, kernel_size=(1

# Calculate mAP

In [32]:
cell2box_mask = torch.zeros((SY, SX, 2))
for i in range(SY):
    for j in range(SX):
        cell2box_mask[i,j,0] = j
        cell2box_mask[i,j,1] = i 

In [44]:
def calculate_mAP(thres, iou_nms, iou_map, loader, model, mask):
    pred_boxes, target_boxes = get_bboxes(loader=loader, 
                                      model=model,
                                      SX=SX,
                                      SY=SY,
                                      B=B,
                                      C=C,
                                      mask=mask,
                                      iou_threshold=iou_nms, 
                                      threshold=thres,
                                      device=DEVICE,
                                      box_format="midpoint")

    mean_avg_prec, avg_prec, cls_prec, cls_rec = mAP(pred_boxes=pred_boxes, 
                                                     true_boxes=target_boxes, 
                                                     iou_threshold=iou_map, 
                                                     box_format="midpoint",
                                                     num_classes=C)
    # print(f'mAP: {mean_avg_prec}')
    # print(f'smoke AP: {avg_prec[0]}')
    # print(f'fire AP: {avg_prec[1]}')
    # print(f'smoke precision: {cls_prec[0]}')
    # print(f'fire precision: {cls_prec[1]}')
    # print(f'smoke recall: {cls_rec[0]}')
    # print(f'fire recall: {cls_rec[1]}')

    f1_smoke = 2*(cls_prec[0]*cls_rec[0])/(cls_prec[0]+cls_rec[0])
    f1_fire = 2*(cls_prec[1]*cls_rec[1])/(cls_prec[1]+cls_rec[1])
    f1_mean = (f1_smoke + f1_fire) / 2

    print("mAP".ljust(9) + "|" + 
          "AP smoke".ljust(9) + "|" + 
          "AP fire".ljust(9) + "|" + 
          "F1 mean".ljust(9) + "|" + 
          "F1 smoke".ljust(9) + "|" + 
          "F1 fire".ljust(9) + "|" +
          "Pr smoke".ljust(9) + "|" +
          "Pr fire".ljust(9) + "|" +
          "Re smoke".ljust(9) + "|" +
          "Re fire".ljust(9))

    print_hyphen = ""
    for i in range(10): # 10 = # of metrics
        print_hyphen += "---------".ljust(9) + " "
    print(print_hyphen)
    
    print(f'{mean_avg_prec:.3f}'.ljust(9) + "|" +
          f'{avg_prec[0]:.3f}'.ljust(9) + "|" +
          f'{avg_prec[1]:.3f}'.ljust(9) + "|" +
          f'{f1_mean:.3f}'.ljust(9) + "|" +
          f'{f1_smoke:.3f}'.ljust(9) + "|" +
          f'{f1_fire:.3f}'.ljust(9) + "|" +
          f'{cls_prec[0]:.3f}'.ljust(9) + "|" +
          f'{cls_prec[1]:.3f}'.ljust(9) + "|" +
          f'{cls_rec[0]:.3f}'.ljust(9) + "|" +
          f'{cls_rec[1]:.3f}'.ljust(9) + "\n")


In [45]:
calculate_mAP(thres=0.2,
              iou_nms=0.3,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/83 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.432    |0.355    |0.509    |0.562    |0.518    |0.606    |0.541    |0.572    |0.496    |0.645    



In [46]:
calculate_mAP(thres=0.2,
              iou_nms=0.5,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/83 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.438    |0.359    |0.517    |0.553    |0.512    |0.594    |0.516    |0.541    |0.507    |0.659    



In [47]:
calculate_mAP(thres=0.1,
              iou_nms=0.5,
              iou_map=0.5,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/83 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:02<00:00,  1.37s/it]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.455    |0.383    |0.528    |0.486    |0.449    |0.523    |0.375    |0.424    |0.560    |0.682    



In [48]:
calculate_mAP(thres=0.2,
              iou_nms=0.3,
              iou_map=0.4,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/83 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:01<00:00,  1.02it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.560    |0.469    |0.651    |0.648    |0.601    |0.695    |0.628    |0.655    |0.576    |0.739    



In [49]:
calculate_mAP(thres=0.2,
              iou_nms=0.3,
              iou_map=0.3,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/83 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.650    |0.559    |0.740    |0.706    |0.661    |0.750    |0.691    |0.708    |0.633    |0.798    



In [50]:
calculate_mAP(thres=0.1,
              iou_nms=0.5,
              iou_map=0.4,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/83 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:02<00:00,  1.36s/it]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.589    |0.507    |0.672    |0.560    |0.524    |0.596    |0.438    |0.483    |0.654    |0.777    



In [51]:
calculate_mAP(thres=0.1,
              iou_nms=0.5,
              iou_map=0.3,
              loader=val_loader,
              model=model,
              mask=cell2box_mask)

Get Boxes:   0%|          | 0/83 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
mAP:@.5: 100%|██████████| 2/2 [00:02<00:00,  1.42s/it]

mAP      |AP smoke |AP fire  |F1 mean  |F1 smoke |F1 fire  |Pr smoke |Pr fire  |Re smoke |Re fire  
--------- --------- --------- --------- --------- --------- --------- --------- --------- --------- 
0.683    |0.606    |0.761    |0.611    |0.580    |0.641    |0.484    |0.520    |0.724    |0.837    

